### You can run the following chunk of code to import any useful libraries and packages needed to run the rest of the script.

In [ ]:
import os as os
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv2
import tensorflow as tf
from scipy.ndimage import label
from skimage import measure
import pandas as pd
from tensorflow.keras.applications.vgg16 import preprocess_input

### You can run the following chunk of code to count cells in MBM images or MBM video frames using a phase 2 classification network.

The user will need to adjust the base_Directory variable, as well as pay careful attention to the directory structure.

In [ ]:
base_Directory = ""


output_Data_Frame = pd.DataFrame(columns = ["File Name", "Counts"])
channel_Directory = base_Directory + "Analysis/All_Images_For_Analysis/"
phase_One_Network_Path = base_Directory + "Training/Phase_1/Trained_Model/Phase_One_Trained_Network.h5"
phase_Two_Network_Path = base_Directory + "Training/Phase_2/Trained_Model/Phase_Two_Trained_Network.h5"

confidence_Values = [0.80]

network = tf.keras.models.load_model(phase_One_Network_Path)
network_Two = tf.keras.models.load_model(phase_Two_Network_Path)

def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

def recolor_Image(img):
    maximum = np.mean(img) + 4*np.std(img)
    minimum = np.mean(img) - 4*np.std(img)
    color_Adjusted_Image = (img - minimum)/(maximum - minimum)
    color_Adjusted_Image[color_Adjusted_Image < 0] = 0
    color_Adjusted_Image[color_Adjusted_Image > 1] = 1
    return color_Adjusted_Image

cell_Capture_Range = 40
frame_Count = 0
for image_Name in os.listdir(channel_Directory):
    count = 0
    frame_Count = frame_Count + 1 
    
    print("Analyzing " + image_Name[:-4])
    full_Channel = plt.imread(channel_Directory + image_Name)
    if np.max(full_Channel) == int(np.max(full_Channel)) and len(str(np.max(full_Channel))) == len(str(int(np.max(full_Channel)))):
        full_Channel = full_Channel.copy()/255.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel.copy(), cv2.COLOR_GRAY2RGB)
    if np.shape(full_Channel)[2] == 4:
        full_Channel = full_Channel.copy()[:,:,0:3]
    full_Channel = recolor_Image(full_Channel.copy())
    image_Height, image_Width, channels = np.shape(full_Channel)
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))
    x_Slider = 0
    y_Slider = 0
    output_Array = np.zeros((128,128))
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            output = network.predict(current_Tile_Normalized, verbose = 0)

            for i in range(128):
                for j in range(128):
                    output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    for i in range(image_Height_Resized):
        for j in range(image_Width_Resized):
            if output_Image[i,j] != 0:
                output_Image[i,j] = 1
            else:
                continue

    blobs, number_Of_Blobs = label(output_Image)
    properties = measure.regionprops(blobs)
    

    for confidence in confidence_Values:
          count = 0
          for prop in properties:
                if round(prop.centroid[0]) < cell_Capture_Range:
                    if round(prop.centroid[1]) < cell_Capture_Range:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[0:cell_Capture_Range,0:cell_Capture_Range]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))
                    elif image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[0:cell_Capture_Range,image_Width_Resized - cell_Capture_Range:image_Width_Resized]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))   
                    else:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[0:cell_Capture_Range,int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))
                elif round(prop.centroid[1]) < cell_Capture_Range:
                    if image_Height_Resized - prop.centroid[0] < cell_Capture_Range:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,0:cell_Capture_Range]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))
                    else:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),0:cell_Capture_Range]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))
                elif image_Height_Resized - prop.centroid[0] < cell_Capture_Range:
                    if image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,image_Width_Resized - cell_Capture_Range:image_Width_Resized]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))
                    else:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[image_Height_Resized - cell_Capture_Range:image_Height_Resized,int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))              
                elif image_Width_Resized - prop.centroid[1] < cell_Capture_Range:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),image_Width_Resized - cell_Capture_Range:image_Width_Resized]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))            
                else:
                        region = np.reshape(preprocess_input((255*full_Channel_Resized[int(round(prop.centroid[0]) - cell_Capture_Range/2):int(round(prop.centroid[0]) + cell_Capture_Range/2),int(round(prop.centroid[1]) - cell_Capture_Range/2):int(round(prop.centroid[1]) + cell_Capture_Range/2)]).astype(int)),(1,cell_Capture_Range,cell_Capture_Range,3))   
                prediction = network_Two.predict(region, verbose = 0)
                if np.argmax(prediction) == 0 and prediction[0,np.argmax(prediction)] >= confidence:
                    count += 1
    output_Data_Frame = pd.concat([output_Data_Frame.copy(), pd.DataFrame([[image_Name,count]], columns = ["File Name", "Counts"])])
    display(output_Data_Frame)
display(output_Data_Frame)

### You can run the following chunk of code to save the counts to a .csv file for later analysis or use.

In [ ]:
output_Data_Frame.to_csv(base_Directory + "/Analysis/Automated_Counts/Counts.csv")